In [2]:
import requests
import folium
from IPython.display import display, HTML, clear_output
import webbrowser
from folium import plugins
import requests
import os
import json
from IPython.display import display, HTML
from dotenv import load_dotenv, find_dotenv

In [ ]:
# !pip install --upgrade openai

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
print("plant diseases",rec_plant_diseases("https://cdn.shopify.com/s/files/1/0579/7924/0580/t/18/assets/buy-mint-plants-step-4_1.jpg"))        #pass the image url 
print("recognize (flowers, bushes, fungi, cacti, succulents, and mushrooms) \n",recognize_plants("https://dukaan.b-cdn.net/700x700/webp/730950/570a0a01-8039-4174-b518-9bf54edfe5fb/img-0348-4f923a91-eb5f-4685-9252-067786e91346.JPEG"))          #pass the image url
print("recognize poisonous plant bases on location \n",poisonous_plant("new york park"))     #pass the location str:("nigeria")
print("plant and flower recommendation \n",recommendation("https://watermark.lovepik.com/photo/20211122/large/lovepik-the-balcony-terrace-is-photographed-in-real-life-picture_500685494.jpg"))              #pass the image url


plant diseases [' {{\n    "plant_disease": "No plant diseases found. Your plant looks healthy.",\n    "solution": "None"\n}}']
recognize (flowers, bushes, fungi, cacti, succulents, and mushrooms) 
 [' ```json\n{\n  "This is": "succulent",\n  "succulent name": "echeveria"\n}\n```']
recognize poisonous plant bases on location 
 ['{\n    "poisonous_plant": "Poison Ivy (Toxicodendron radic', 'ans)",\n    "risks": "Exposure to Poison Ivy can cause severe skin irritation, characterized by intense itching, redness, and blistering. It contains urush', 'iol, an oily resin that triggers an allergic reaction upon contact with human skin.",\n    "safety_recommendations": "To ensure safety, follow these steps:\n                              1. Avoid contact with Poison Ivy, especially during the spring and summer months when the plant is actively growing.\n                              2. If contact occurs,', ' promptly wash the affected area with soap and water.\n                              3. A

In [5]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

# _ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
# Map available functions
available_functions = {
    "get_plant_diseases": rec_plant_diseases,
    "get_recognize_plants": recognize_plants,
    "get_poisonous_plant": poisonous_plant,
    "get_recommendation": recommendation
}

In [7]:
bot_instructions = """your task is to recognize plant diseases, poisonous_plant, flower and plants recommendation and 
identify flowers, bushes, fungi, cacti, succulents, and mushrooms
We will provide APIs for all plant diseases, poisonous_plant, flower and plants recommendation and 
identify (flowers, bushes, fungi, cacti, succulents, and mushrooms.). You will receive the necessary information from the API response."

if any other agriculture Qurey you can answer them. "If user asks a question about anything other than agriculture, respond with: 'This is an agriculture chatbot. Feel free to ask any questions about agriculture!'" 
"""

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_plant_diseases",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_url": {"type": "string"},
                },
                "required": ["image_url"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_recognize_plants",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_url": {"type": "string"},
                },
                "required": ["image_url"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_poisonous_plant",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                },
                "required": ["location"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_recommendation",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_url": {"type": "string"},
                },
                "required": ["image_url"],
            }
        }
    },
]

In [16]:
# A Class to Manage All Open API Assistant Calls and Functions
from openai.types.beta.threads import Run, ThreadMessage
from openai.types.beta.thread import Thread
from openai.types.beta.assistant_create_params import Tool

import time

class AgricultureAssistantManager:
    def __init__(self, model: str = "gpt-3.5-turbo-1106"):
        self.client = OpenAI()
        self.model = model
        self.assistant = None
        self.thread = None
        self.run = None

    def create_assistant(self, name: str, instructions: str, tools: list[Tool]) -> None:
        self.assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            tools=tools,
            model=self.model
        )

    def create_thread(self) -> Thread:
        self.thread = self.client.beta.threads.create()
        return self.thread

    def add_message_to_thread(self, role: str, content: str) -> None:
        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role=role,
            content=content
        )

    def run_assistant(self, instructions: str) -> Run:
        self.run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
            instructions=instructions
        )
        return self.run

    def wait_for_completion(self, run: Run, thread: Thread) -> Run:

        while run.status in ["in_progress", "queued"]:
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=self.run.id
            )
            print(f"Run is {run.status}. Waiting...")
            time.sleep(3)  # Wait for 3 seconds before checking again

            if run_status.status == 'completed':
                processed_response = self.process_messages()
                return processed_response
                # break
            elif run_status.status == 'requires_action':
                print("Function Calling ...")
                self.call_required_functions(run_status.required_action.submit_tool_outputs.model_dump())
            elif run.status == "failed":
                print("Run failed.")
                break
            else:
                print(f"Waiting for the Assistant to process...: {run.status}")

    def process_messages(self) -> list[ThreadMessage]:
        messages: list[ThreadMessage] = self.client.beta.threads.messages.list(thread_id=self.thread.id)
        return messages

    def call_required_functions(self, required_actions: dict):
        tool_outputs = []

        for action in required_actions["tool_calls"]:
            function_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            print('function_name', function_name)
            print('function_arguments', arguments)

            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                output = function_to_call(**arguments)

                # Serialize output to a string if it's not already one
                if not isinstance(output, str):
                    output = json.dumps(output)

                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output,  # This is now ensured to be a string
                })

            else:
                raise ValueError(f"Unknown function: {function_name}")

        print("Submitting outputs back to the Assistant...")
        self.client.beta.threads.runs.submit_tool_outputs(
            thread_id=self.thread.id,
            run_id=self.run.id,
            tool_outputs=tool_outputs
        )


In [17]:
# Show Messages and Plot Images in Financial Analysis If ANY

import requests
from PIL import Image
from IPython.display import Image, display


def download_and_save_image(file_id: str, save_path: str) -> None:
    """
    Downloads an image from OpenAI using its file ID and saves it to the specified path.

    Args:
    - file_id (str): The ID of the file to download.
    - save_path (str): The path where the image will be saved.

    Returns:
    - None
    """
    # Construct the URL to download the image
    download_url = f"https://api.openai.com/v1/files/{file_id}/content"

    # Perform the HTTP GET request to download the image
    response = requests.get(download_url, headers={"Authorization": f'Bearer {os.getenv("OPENAI_API_KEY")}'})

    # Check if the request was successful
    if response.status_code == 200:
        # Write the image to the specified file
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image downloaded and saved to {save_path}")
    else:
        print(f"Failed to download image: HTTP Status Code {response.status_code}")


def pretty_print(messages: list[ThreadMessage]) -> None:
    print("# Messages")
    for message in messages.data:
        role_label = "User" if message.role == "user" else "Assistant"
        # Check the type of message content and handle accordingly
        for content in message.content:
            if content.type == "text":
                message_content = content.text.value
                print(f"{role_label}: {message_content}\n")
                print()
            elif content.type == "image_file":
                # Handle image file content, e.g., print the file ID or download the image
                image_file_id = content.image_file.file_id
                print(f"{role_label}: Image file ID: {image_file_id}")
                # Define a path to save the image
                image_save_path = f"image_{image_file_id}.png"
                # Download and save the image
                # print(f"{role_label}: Image file ID: {image_file_id}")
                download_and_save_image(image_file_id, image_save_path)

                # Display the image within Jupyter Notebook
                display(Image(filename=image_save_path))

                #   # Open and display the image
                # try:
                #     img = Image.open(image_save_path)
                #     img.show()
                # except IOError:
                #     print("Error in opening the image file.")

In [18]:
def fmp_financial_analyst(prompt: str):
    fmp_analyst = AgricultureAssistantManager()

    fmp_analyst.create_assistant(
        name="AgricultureAssistant",
        instructions=bot_instructions,
        tools=tools
    )

    fmp_analyst.create_thread()

    fmp_analyst.add_message_to_thread(
        role="user",
        content=prompt
    )

    run = fmp_analyst.run_assistant(
        instructions=bot_instructions
    )

    final_res = fmp_analyst.wait_for_completion(
        run=run,
        thread=fmp_analyst.thread
    )

    return final_res
    

In [19]:
# analysis1 = fmp_financial_analyst("Can you compare the financial health of Microsoft and Apple over the last years, focusing on their balance sheets and key financial ratios?")

In [20]:
# pretty_print(analysis1)

## Visualize Microsoft vs. Googles's revenue & profitability

In [21]:
response2 = fmp_financial_analyst("""
which plant do you see in this image?
"https://dukaan.b-cdn.net/700x700/webp/730950/570a0a01-8039-4174-b518-9bf54edfe5fb/img-0348-4f923a91-eb5f-4685-9252-067786e91346.JPEG"
""")

Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...
Function Calling ...
function_name get_recognize_plants
function_arguments {'image_url': 'https://dukaan.b-cdn.net/700x700/webp/730950/570a0a01-8039-4174-b518-9bf54edfe5fb/img-0348-4f923a91-eb5f-4685-9252-067786e91346.JPEG'}
Submitting outputs back to the Assistant...
Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...


In [22]:
pretty_print(response2)

# Messages
Assistant: The plant in the image is a succulent, specifically an Echeveria. If you have any more questions about succulents or any other plant-related queries, feel free to ask!


User: 
which plant do you see in this image?
"https://dukaan.b-cdn.net/700x700/webp/730950/570a0a01-8039-4174-b518-9bf54edfe5fb/img-0348-4f923a91-eb5f-4685-9252-067786e91346.JPEG"





In [23]:
response2 = fmp_financial_analyst("""
poisonous plant in new york park
""")

Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...
Function Calling ...
function_name get_poisonous_plant
function_arguments {'location': 'new york park'}
Submitting outputs back to the Assistant...
Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...


In [24]:
response3 = fmp_financial_analyst("""
what is Rust plant diseases describe in 10 words
""")

Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...
Function Calling ...
function_name get_plant_diseases
function_arguments {'image_url': 'https://www.almanac.com/sites/default/files/image_nodes/tomato-rust.jpg'}
Submitting outputs back to the Assistant...
Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...


In [25]:
pretty_print(response2)

# Messages
Assistant: The poisonous plant found in New York parks is Poison Ivy (Toxicodendron radicans). Exposure to Poison Ivy can cause severe skin irritation, known as contact dermatitis. Symptoms include redness, itching, swelling, and blisters. The rash can be extremely uncomfortable and can take several weeks to heal. Children are at particular risk because their skin is more sensitive and they may not be aware of the dangers of the plant.

Safety Recommendations:
1. Learn to identify Poison Ivy and other poisonous plants common in your area.
2. Wear long sleeves, pants, and gloves when working in areas where Poison Ivy may be present.
3. If you come into contact with Poison Ivy, wash the area thoroughly with soap and water as soon as possible.
4. Apply a calamine lotion or hydrocortisone cream to help relieve itching and inflammation.


User: 
poisonous plant in new york park





In [26]:
pretty_print(response3)

# Messages
Assistant: I apologize, but it seems that I encountered an issue while trying to retrieve information on Rust plant diseases. Please feel free to ask about any other plant diseases, or let me know if there's anything else I can assist you with!


User: 
what is Rust plant diseases describe in 10 words





In [19]:
response2 = fmp_financial_analyst("brif explain about financial modeling prep API explain in max 50 words")

Run is queued. Waiting...
Waiting for the Assistant to process...: queued
Run is queued. Waiting...


In [20]:
pretty_print(response2)

# Messages
Assistant: The Financial Modeling Prep API provides access to comprehensive financial data, including income statements, balance sheets, cash flow statements, key metrics, financial ratios, and growth trends. It allows analysts to retrieve, analyze, and integrate fundamental financial data for various stocks and companies.


User: brif explain about financial modeling prep API explain in max 50 words




ask Question reather then function calling